In [1]:
# Data visualization
import numpy as np
import pandas as pd 

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, TensorBoard
import keras_tuner as kt
from sklearn.metrics import accuracy_score
from keras.optimizers import RMSprop


# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "RIGHT_WRIST",
    "LEFT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [3]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

In [4]:
# load dataset
df = describe_dataset("./train.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0

df.loc[df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 5836 
Number of columns: 37

Labels: 
C    3175
L    2661
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [5]:
with open("./model/input_scaler.pkl", "rb") as f:
    sc = pickle.load(f)

In [6]:
# Standard Scaling of features
x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

Model

In [8]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)

# Final Results
final_models = {}

In [9]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

Model - 3 Layers

In [10]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model




In [11]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo',
)
tuner_3l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 03s]
val_accuracy: 0.9863013625144958

Best val_accuracy So Far: 0.994006872177124
Total elapsed time: 00h 00m 44s
INFO:tensorflow:Oracle triggered exit


In [12]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x0000017D94D74A60>
Layer-2: 160 units, func:  <function relu at 0x0000017D94D74A60>
Layer-3: 2 units, func:  <function softmax at 0x0000017D94D74040>

Other params:
learning_rate: 0.001
Epoch 1/100
467/467 [==============================] - 1s 1ms/step - loss: 0.1911 - accuracy: 0.9250 - val_loss: 0.1022 - val_accuracy: 0.9623
Epoch 2/100
467/467 [==============================] - 0s 1000us/step - loss: 0.0678 - accuracy: 0.9820 - val_loss: 0.0826 - val_accuracy: 0.9649
Epoch 3/100
467/467 [==============================] - 0s 1ms/step - loss: 0.0447 - accuracy: 0.9867 - val_loss: 0.0732 - val_accuracy: 0.9769
Epoch 4/100
467/467 [==============================] - 0s 1ms/step - loss: 0.0379 - accuracy: 0.9889 - val_loss: 0.0557 - val_accuracy: 0.9795
Epoch 5/100
467/467 [==============================] - 0s 1ms/step - loss: 0.0298 - accuracy: 0.9916 - val_loss: 0.0442 - val_accuracy: 0.9863
Epoch 6/100
467/467 [=

In [13]:
model_3l.save("./model/3LayeredModelCorrected.h5")

In [14]:
final_models["3_layers"] = model_3l

Model With 5 Layers

In [15]:
# def model_5l_builder(hp):
#     model = Sequential()
#     model.add(Dense(36, input_dim = 36, activation = "relu"))

#     hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
#     hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
#     hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
#     hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
#     hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

#     model.add(Dense(units=hp_layer_1, activation=hp_activation))
#     model.add(Dense(units=hp_layer_2, activation=hp_activation))
#     model.add(Dense(units=hp_layer_3, activation=hp_activation))
#     model.add(Dense(2, activation = "softmax"))

#     model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
#     return model

In [17]:
# tuner_5l = kt.Hyperband(
#     model_5l_builder,
#     objective='val_accuracy',
#     max_epochs=10,
#     directory='keras_tuner_dir',
#     project_name='keras_tuner_demo_2'
# )
# tuner_5l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

In [18]:
# model_5l = get_best_model(tuner_5l)
# model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

In [ ]:
# final_models["5_layers"] = model_5l

In [ ]:
# model_5l.save("./model/5LayeredModelCorrected.h5")

In [ ]:
# with open("./model/all_models.pkl", "wb") as f:
#     pickle.dump(final_models, f)
    

In [19]:
def hybrid_model_builder(hp):
    model = Sequential()
    model.add(Dense(72, input_dim=36, activation='relu')) # Increase the number of neurons in the first layer
    
    # Add two additional dense layers
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    
    # Use a different optimizer and add dropout regularization
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop'])
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    
    if hp_optimizer == 'adam':
        optimizer = Adam(learning_rate=hp_learning_rate)
    else:
        optimizer = RMSprop(learning_rate=hp_learning_rate)
    
    model.add(Dropout(rate=hp_dropout))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [20]:
from tensorflow.keras.layers import BatchNormalization


def hybrid_model2_builder(hp):
    model = Sequential()
    model.add(Dense(72, input_dim=36, activation='relu')) # Increase the number of neurons in the first layer
    
    # Add five additional dense layers with batch normalization and dropout
    hp_layer = [hp.Int('layer_' + str(i), min_value=32, max_value=512, step=32) for i in range(1, 6)]
    hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    for i in range(5):
        model.add(Dense(units=hp_layer[i], activation=hp_activation))
        model.add(BatchNormalization())
        hp_dropout = hp.Float('dropout_' + str(i), min_value=0.0, max_value=0.5, step=0.1)
        model.add(Dropout(rate=hp_dropout))
    
    # Use a different optimizer
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd', 'Adagrad'])
    
    if hp_optimizer == 'adam':
        optimizer = Adam(learning_rate=hp_learning_rate)
    elif hp_optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=hp_learning_rate)
    elif hp_optimizer == 'sgd':
        optimizer = SGD(learning_rate=hp_learning_rate)
    else:
        optimizer = Adagrad(learning_rate=hp_learning_rate)
    
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [21]:
tuner_hybrid = kt.Hyperband(
    hybrid_model_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='hybrid_model_tuner'
)

tuner_hybrid.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard('./keras_tuner_dir/logs')])


INFO:tensorflow:Reloading Tuner from keras_tuner_dir\hybrid_model_tuner\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [22]:
tuner_hybrid2 = kt.Hyperband(
    hybrid_model2_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='hybrid_model_tuner'
)

tuner_hybrid2.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard('./keras_tuner_dir/logs')])


INFO:tensorflow:Reloading Tuner from keras_tuner_dir\hybrid_model_tuner\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [23]:
best_hybrid_model = get_best_model(tuner_hybrid)
best_hybrid_model.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])



Describe models architecture
Layer-1: 72 units, func:  <function relu at 0x0000017D94D74A60>
Layer-2: 288 units, func:  <function relu at 0x0000017D94D74A60>
Layer-3: 192 units, func:  <function relu at 0x0000017D94D74A60>
Layer-4: 448 units, func:  <function relu at 0x0000017D94D74A60>
Layer-5: 192 units, func:  <function relu at 0x0000017D94D74A60>
Layer-6: 512 units, func:  <function relu at 0x0000017D94D74A60>
Layer-7: 0 units, func: None
Layer-8: 2 units, func:  <function softmax at 0x0000017D94D74040>

Other params:
learning_rate: 0.0001
optimizer: adam
dropout: 0.1
Epoch 1/100
467/467 [==============================] - 2s 3ms/step - loss: 0.2168 - accuracy: 0.9068 - val_loss: 0.1027 - val_accuracy: 0.9580
Epoch 2/100
467/467 [==============================] - 1s 2ms/step - loss: 0.0695 - accuracy: 0.9741 - val_loss: 0.0885 - val_accuracy: 0.9692
Epoch 3/100
467/467 [==============================] - 1s 2ms/step - loss: 0.0469 - accuracy: 0.9839 - val_loss: 0.0550 - val_accuracy:

In [24]:
best_hybrid_model2 = get_best_model(tuner_hybrid2)
best_hybrid_model2.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])


Describe models architecture
Layer-1: 72 units, func:  <function relu at 0x0000017D94D74A60>
Layer-2: 288 units, func:  <function relu at 0x0000017D94D74A60>
Layer-3: 0 units, func: None
Layer-4: 0 units, func: None
Layer-5: 192 units, func:  <function relu at 0x0000017D94D74A60>
Layer-6: 0 units, func: None
Layer-7: 0 units, func: None
Layer-8: 448 units, func:  <function relu at 0x0000017D94D74A60>
Layer-9: 0 units, func: None
Layer-10: 0 units, func: None
Layer-11: 192 units, func:  <function relu at 0x0000017D94D74A60>
Layer-12: 0 units, func: None
Layer-13: 0 units, func: None
Layer-14: 512 units, func:  <function relu at 0x0000017D94D74A60>
Layer-15: 0 units, func: None
Layer-16: 0 units, func: None
Layer-17: 2 units, func:  <function softmax at 0x0000017D94D74040>

Other params:
learning_rate: 0.0001
optimizer: adam
dropout: 0.1
dropout_0: 0.0
dropout_1: 0.0
dropout_2: 0.0
dropout_3: 0.0
dropout_4: 0.0
Epoch 1/100
467/467 [==============================] - 3s 3ms/step - loss: 0.

In [ ]:
best_hybrid_model.save("./model/hybrid_model_Corrected.h5")
best_hybrid_model2.save("./model/hybrid_model_Corrected2.h5")